In [1]:
# install dependencies: (use cu101 because colab has CUDA 10.1)
!pip install -U torch==1.5 torchvision==0.6 -f https://download.pytorch.org/whl/cu101/torch_stable.html 
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab

Looking in links: https://download.pytorch.org/whl/cu101/torch_stable.html
Requirement already up-to-date: torch==1.5 in /usr/local/lib/python3.6/dist-packages (1.5.0+cu101)
Requirement already up-to-date: torchvision==0.6 in /usr/local/lib/python3.6/dist-packages (0.6.0+cu101)
  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-6ipqx6k7
  Running command git clone -q https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-6ipqx6k7
  Created wheel for pycocotools: filename=pycocotools-2.0-cp36-cp36m-linux_x86_64.whl size=266462 sha256=f38ca1966c4438d17695caab7e0b044045761e3d806e86a373d1dda9408d9aef
  Stored in directory: /tmp/pip-ephem-wheel-cache-kpty241b/wheels/90/51/41/646daf401c3bc408ff10de34ec76587a9b3ebfac8d21ca5c3a
Successfully built pycocotools
  Found existing installation: pycocotools 2.0
    Uninstalling pycocotools-2.0:
      Successfully uninstalled pycocotools-2.0
1.5.0+cu101 True
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free

In [2]:
# mount your gdrive folder
from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%cd /content/drive/My Drive/ML_Application/freeze_at2_1000_samples


/content/drive/My Drive/ML_Application/freeze_at2_1000_samples


In [4]:
# install detectron2:
!pip install detectron2==0.1.3 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.5/index.html

Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.5/index.html


In [5]:
# You may need to restart your runtime prior to this, to let your installation take effect
# Some basic setup:
# Setup detectron2 logger

import detectron2
from detectron2.utils.logger import setup_logger
#setup_logger()

# import some common libraries
import numpy as np
import cv2
import random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities

from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

In [6]:
import os
import datetime
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg

# create the output dir for the trained model
output_dir = os.path.join("/content/drive/My Drive/ML_Application/freeze_at2_1000_samples/output_freeze2", "20200804T1348") #20200715T0954
# change the config in the .yaml document
print(output_dir)

/content/drive/My Drive/ML_Application/freeze_at2_1000_samples/output_freeze2/20200804T1348


In [7]:

# Register a dataset in COCO’s json annotation format for instance detection, instance segmentation and keypoint detection. 
from detectron2.data.datasets import register_coco_instances
register_coco_instances("my_dataset_train", {}, "/content/drive/My Drive/ML_Application/freeze_at2_1000_samples/data/train/coco_train.json", "/content/drive/My Drive/ML_Application/freeze_at2_1000_samples/data/train")
register_coco_instances("my_dataset_val", {}, "/content/drive/My Drive/ML_Application/freeze_at2_1000_samples/data/val/coco_val.json", "/content/drive/My Drive/ML_Application/freeze_at2_1000_samples/data/val")
register_coco_instances("my_dataset_test", {}, "/content/drive/My Drive/ML_Application/freeze_at2_1000_samples/data/test/coco_test.json", "/content/drive/My Drive/ML_Application/freeze_at2_1000_samples/data/test")

In [8]:
from detectron2.data import DatasetCatalog, MetadataCatalog

papers_metadata = MetadataCatalog.get("my_dataset_train") #return the Metadata instance associated with this name
dataset_dicts = DatasetCatalog.get("my_dataset_train") # Call the registered function and return dataset annotations


Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.



In [10]:
cfg = get_cfg() #load default config
cfg.merge_from_file(output_dir + "/train_config_freeze2.yaml")
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set the testing threshold for this model
# suppress overlap of bounding boxes by applying non-maximum suppression (NMS)
cfg.MODEL.ROI_HEADS.NMS_THRESH_TEST = 0.5
cfg.DATASETS.TEST = ("my_dataset_val", )
predictor = DefaultPredictor(cfg)

In [11]:
predictor

In [15]:
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)

from detectron2.utils.visualizer import ColorMode
dataset_dicts_val = DatasetCatalog.get("my_dataset_val")
for d in dataset_dicts_val:    
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1],
                   metadata=papers_metadata, 
                   scale=0.8, 
                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels
    )
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(v.get_image()[:, :, ::-1])

Output hidden; open in https://colab.research.google.com to view.

In [16]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("my_dataset_val", cfg, False, output_dir=output_dir)
val_loader = build_detection_test_loader(cfg, "my_dataset_val")
inference_on_dataset(trainer.model, val_loader, evaluator)
# another equivalent way is to use trainer.test

WARNING [08/05 07:17:25 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[08/05 07:17:25 d2.data.datasets.coco]: Loaded 183 images in COCO format from /content/drive/My Drive/ML_Application/freeze_at2_1000_samples/data/val/coco_val.json
[08/05 07:17:25 d2.data.build]: Distribution of instances among all 9 categories:
|  category  | #instances   |  category  | #instances   |  category   | #instances   |
|:----------:|:-------------|:----------:|:-------------|:-----------:|:-------------|
|  abstract  | 102          |  address   | 48           | affiliation | 129          |
|   author   | 183          |    date    | 186          |     doi     | 48           |
|   email    | 47           |  journal   | 138          |    title    | 183          |
|            |              |            |              |             |              |
|   total    | 1064         |            |              |             |              |
[08/05 

OrderedDict([('bbox',
              {'AP': 24.517325168767066,
               'AP-abstract': 92.21566320760236,
               'AP-address': 0.0,
               'AP-affiliation': 14.908062234794908,
               'AP-author': 27.45813307851199,
               'AP-date': 10.088072025593364,
               'AP-doi': 0.0,
               'AP-email': 5.693069306930694,
               'AP-journal': 2.9702970297029703,
               'AP-title': 67.32262963576729,
               'AP50': 28.50971707683703,
               'AP75': 26.78507217190422,
               'APl': 36.46487431825471,
               'APm': 28.859280605103976,
               'APs': 3.7370737073707367}),
             ('segm',
              {'AP': 24.122279964323614,
               'AP-abstract': 92.97915481146498,
               'AP-address': 0.0,
               'AP-affiliation': 12.654375556603279,
               'AP-author': 28.04883530463537,
               'AP-date': 8.985622946531107,
               'AP-doi': 0.0,
     

Making predictions on the test set

In [16]:
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)

from detectron2.utils.visualizer import ColorMode
dataset_dicts_test = DatasetCatalog.get("my_dataset_test")

for d in dataset_dicts_test:    
    im = cv2.imread(d["file_name"])
    print(d["file_name"][:-5])
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1],
                   metadata=papers_metadata, 
                   scale=0.8, 
                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels
    )
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2.imwrite(d["file_name"][:-5] + "_test_nms.png", v.get_image()[:, :, ::-1])
    #cv2_imshow(v.get_image()[:, :, ::-1])

[08/05 07:51:37 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

In [17]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("my_dataset_test", cfg, False, output_dir=output_dir)
val_loader = build_detection_test_loader(cfg, "my_dataset_test")
inference_on_dataset(trainer.model, val_loader, evaluator)
# another equivalent way is to use trainer.test

WARNING [08/05 07:54:30 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[08/05 07:54:30 d2.data.datasets.coco]: Loaded 194 images in COCO format from /content/drive/My Drive/ML_Application/freeze_at2_1000_samples/data/test/coco_test.json
[08/05 07:54:30 d2.data.build]: Distribution of instances among all 9 categories:
|  category  | #instances   |  category  | #instances   |  category   | #instances   |
|:----------:|:-------------|:----------:|:-------------|:-----------:|:-------------|
|  abstract  | 183          |  address   | 4            | affiliation | 344          |
|   author   | 307          |    date    | 145          |     doi     | 137          |
|   email    | 2            |  journal   | 86           |    title    | 194          |
|            |              |            |              |             |              |
|   total    | 1402         |            |              |             |              |
[08/0

OrderedDict([('bbox',
              {'AP': 15.79309532428064,
               'AP-abstract': 71.97062024140274,
               'AP-address': 0.0,
               'AP-affiliation': 0.0,
               'AP-author': 7.212214264898999,
               'AP-date': 5.379537953795379,
               'AP-doi': 0.0,
               'AP-email': 0.0,
               'AP-journal': 5.646047912233869,
               'AP-title': 51.92943754619478,
               'AP50': 19.12157953814279,
               'AP75': 17.655336396168618,
               'APl': 26.070536571816668,
               'APm': 7.0731974154346915,
               'APs': 0.7567006700670067}),
             ('segm',
              {'AP': 15.60748171189698,
               'AP-abstract': 72.26712121014616,
               'AP-address': 0.0,
               'AP-affiliation': 0.0,
               'AP-author': 7.090514485368058,
               'AP-date': 5.1815181518151805,
               'AP-doi': 0.0,
               'AP-email': 0.0,
               'AP